In [4]:
from pynvml.smi import nvidia_smi
def get_gpu_watts():
    """ 
    Gets power draw from NVIDIA-smi
    
    Power draw is the average power draw (average over unknown duration) for modern GPUs.
    Old GPUs might show instantaneous power draw, not average. Refer to NVIDIA-smi for more details.
    """
    watts = None
    try:
        nvsmi = nvidia_smi.getInstance()
        result = nvsmi.DeviceQuery("power.draw")
        result = result["gpu"][0]["power_readings"]
        unit = result["unit"]
        watts = result["power_draw"]
        assert unit == "W"  # Should always be watts?
    except Exception as e:
        print(e)
    return watts

# get_gpu_memory()
get_gpu_watts()

205.707

In [5]:
import psutil
import GPUtil
import threading
import time
import pandas as pd

class PsutilCollector:
    def __init__(self, interval=1):
        self.interval = interval
        self.data = []  # List to store collected metrics
        self.running = False
        self.thread = None
        self.clear_requested = False

    def collect_metrics(self):
        while self.running:
            if self.clear_requested:
                self.clear_requested = False
                self.data.clear()
            cpu_percent = psutil.cpu_percent(interval=self.interval)
            GPUs = GPUtil.getGPUs()
            gpu_utilization = GPUs[0].load
            memory_percent = psutil.virtual_memory().percent
            disk_read_bytes = psutil.disk_io_counters().read_bytes
            disk_write_bytes = psutil.disk_io_counters().write_bytes
            network_bytes_sent = psutil.net_io_counters().bytes_sent
            network_bytes_received = psutil.net_io_counters().bytes_recv
            gpu_watts = get_gpu_watts()

            # Append metrics to the list
            self.data.append({
                'CPU Percent': cpu_percent,
                'GPU Percent': gpu_utilization,
                'Memory Percent': memory_percent,
                'Disk Read Bytes': disk_read_bytes,
                'Disk Write Bytes': disk_write_bytes,
                'Network Bytes Sent': network_bytes_sent,
                'Network Bytes Received': network_bytes_received,
                "timestamp": time.time(),
                'GPU Watts': gpu_watts,
            })
            per_cpu = psutil.cpu_percent(interval=self.interval, percpu=True)
            self.data[-1].update({f"cpu_{cpu}": percent for cpu, percent in enumerate(per_cpu)})

    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self.collect_metrics)
        self.thread.start()

    def stop(self):
        self.running = False
        self.thread.join()
        
    def clear_data(self):
        self.clear_requested = True  # Instead of clearing data, set a flag to avoid issues with multithreading

    def create_dataframe(self):
        return pd.DataFrame(self.data)

    def __enter__(self):
        # Code to execute when entering the with-statement
        self.start()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        # Code to execute when exiting the with-statement
        self.stop()

# Example usage
if __name__ == "__main__":
    collector = PsutilCollector(interval=1)  # Set your desired interval
    collector.start()

    # Simulate experiment (replace with your actual experiment logic)
    time.sleep(2)  # Run the experiment for 10 seconds

    collector.stop()
    df = collector.create_dataframe()

    print("Collected metrics:")

    print(df)
    print(df.columns)


Collected metrics:
   CPU Percent  GPU Percent  Memory Percent  Disk Read Bytes  \
0         21.8         0.93            46.9       5839041024   

   Disk Write Bytes  Network Bytes Sent  Network Bytes Received     timestamp  \
0        1840349696            11075200                29686772  1.714043e+09   

   GPU Watts  cpu_0  ...  cpu_6  cpu_7  cpu_8  cpu_9  cpu_10  cpu_11  cpu_12  \
0    208.984   24.5  ...   21.4   24.2   15.8   66.0    18.6    15.2    16.2   

   cpu_13  cpu_14  cpu_15  
0    13.1    17.8     8.3  

[1 rows x 25 columns]
Index(['CPU Percent', 'GPU Percent', 'Memory Percent', 'Disk Read Bytes',
       'Disk Write Bytes', 'Network Bytes Sent', 'Network Bytes Received',
       'timestamp', 'GPU Watts', 'cpu_0', 'cpu_1', 'cpu_2', 'cpu_3', 'cpu_4',
       'cpu_5', 'cpu_6', 'cpu_7', 'cpu_8', 'cpu_9', 'cpu_10', 'cpu_11',
       'cpu_12', 'cpu_13', 'cpu_14', 'cpu_15'],
      dtype='object')


In [6]:
# Init CARLA (func)
from CarlaClient import CarlaClient
import random

class ClientWrapper:
    def __init__(self, img_width, img_height):
        host = 'localhost'
        port = 2000
        img_width = img_width
        img_height = img_height
        fps = 30
        self.img_counter = 0  # Mostly for verifying that sensor data is being received
        
        env = CarlaClient(host, port, img_width, img_height, fps, lidar_points_per_second=1000)
        print("Carla client initialized")
    
        env.set_sync_mode()
        print("Carla client synchronized")
        
        #env.spawn_pedestrians(50)
        #env.move_pedestrians()
        self.spawn_points = env.world.get_map().get_spawn_points()
    
        self.vehicles = []
        self.cameras = []
        random.seed(1)  # Set seed for repeatability
        self.env = env
        
    def tick(self):
        self.env.world.tick()
        
    def sensor_callback(self, data):
        self.img_counter += 1
        if self.img_counter % 500 == 0:
            print("Got image %d" % self.img_counter)
    
    def close_carla(self):
        self.env.clear_actors()
        self.tick()
        self.env.disable_sync_mode()
        # TODO: How to actually close this connection?

    # Spawn vehicles and sensors (func)
    ## -- mostly used incrementally to speed up testing
    ## -- needs spawn collision detection (reattempts and finally a failure handler)

    def add_vehicle(self):
        vehicle_model = 'vehicle.tesla.model3'  # Carla also has a method for random cars if it matters
        done = False
        while not done:
            spawn_point = random.randint(0, len(self.spawn_points))
            try:
                vehicle = self.env.spawn_vehicle(vehicle_model, spawn_point)
                done = True
            except:
                print("Spawn occupied - attempting again...")
                done = False
    
        self.vehicles.append(vehicle)
        vehicle_id = len(self.vehicles)
    
        sensor_name = f'camera_{vehicle_id}'
        camera = self.env.spawn_camera((0, 0, 2), vehicle=vehicle)
        print("Spawn camera %d" % vehicle_id)
        camera.listen(lambda image: self.sensor_callback(image))
        #self.env.add_sensor(sensor_name)
        self.env.camera_sensor_list.append(camera)
        self.cameras.append(camera)
        self.env.set_autopilot()
    
    def remove_vehicle(self):
        # env.remove_vehicle()
        # vehicles.pop()
        # cameras.pop()
        vehicle = self.vehicles.pop()
        sensor = self.cameras.pop()
        sensor.destroy()
        vehicle.destroy()
        self.env.vehicle_list.remove(vehicle)
        self.env.camera_sensor_list.remove(sensor)

    def __enter__(self):
        # Code to execute when entering the with-statement
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        # Code to execute when exiting the with-statement
        self.close_carla()



In [7]:
# For testing the carla client
with ClientWrapper(640,640) as client:
    client.add_vehicle()
    client.tick()



Carla client initialized
Carla client synchronized
Spawn camera 1
Destroyed actor
Could not destroy actor
Destroyed actor


In [9]:
import os
# Measurement loop
## -- Add sensor
## -- Measure
## -- Save data (Account for restarts, failures and repeats -> but may mess up the psutil metrics)
## -- Repeat
import pandas as pd
import pyRAPL

pyRAPL.setup()


result_rows = []
# sensor_width = 640
# sensor_height = 640
max_frames = 1000
temp_folder = f"outputs/run_3/intermediate"
os.makedirs(temp_folder, exist_ok=True)
with PsutilCollector(interval=1) as collector:
    for sensor_width, sensor_height in ((160,160), (320,320), (640,640), (1280,1280)):
        print(f"Running with {sensor_width}x{sensor_height}")
        with ClientWrapper(sensor_width, sensor_height) as client:
            time.sleep(5) # Let psutil collector collect the background HW utilization for reference
            for num_cameras in range(1, 15, 1):
                output_name = f"{sensor_width}x{sensor_height}-{num_cameras}"
                output_name_carla = f"{temp_folder}/carla_{output_name}.feather"
                output_name_psutil = f"{temp_folder}/psutil_{output_name}.feather"
                if os.path.exists(output_name_carla) and os.path.exists(output_name_psutil):
                    print(f"Skipping {output_name}")
                    continue
                while len(client.cameras) < num_cameras:
                    client.add_vehicle()
                measure = pyRAPL.Measurement('bar')
                measure.begin()
                client.tick()  # Warmup 
                start_time = time.time()
                for frame in range(max_frames):
                    client.tick()
                end_time = time.time()
                measure.end()
                rapl_joules = measure.result.pkg
                rapl_duration = measure.result.duration
                print(f"{rapl_joules} joules")
                result_rows.append({"num_cameras": num_cameras, "num_frames": max_frames, 
                                    "exec_time": end_time - start_time, "start_timestamp": start_time, "end_time": end_time,
                                    "rapl_joules": rapl_joules, "rapl_duration": rapl_duration,
                                    "sensor_w": sensor_width, "sensor_h": sensor_height, "sensor_size": f"{sensor_width}x{sensor_height}"})
                
                # Create intermediary DFs as a workaround for potential crashing or freezing
                psutil_df = collector.create_dataframe()
                collector.clear_data()
                df = pd.DataFrame(result_rows)
                psutil_df.to_feather(output_name_psutil)
                df.to_feather(output_name_carla)
                result_rows = []  # Reset data, as we are saving a new file after each iteration

print("DONE")


Running with 160x160
Carla client initialized
Carla client synchronized
Spawn camera 1
Got image 500
Got image 1000
[249870271.0] joules
Spawn camera 2
Got image 1500
Got image 2000
Got image 2500
Got image 3000
[322935697.0] joules
Spawn camera 3
Got image 3500
Got image 4000
Got image 4500
Got image 5000
Got image 5500
Got image 6000
[415153173.0] joules
Spawn camera 4
Got image 6500
Got image 7000
Got image 7500
Got image 8000
Got image 8500
Got image 9000
Got image 9500
Got image 10000
[495648573.0] joules
Spawn camera 5
Got image 10500
Got image 11000
Got image 11500
Got image 12000
Got image 12500
Got image 13000
Got image 13500
Got image 14000
Got image 14500
Got image 15000
[552340322.0] joules
Spawn camera 6
Got image 15500
Got image 16000
Got image 16500
Got image 17000
Got image 17500
Got image 18000
Got image 18500
Got image 19000
Got image 19500
Got image 20000
Got image 20500
Got image 21000
[666394474.0] joules
Spawn camera 7
Got image 21500
Got image 22000
Got image 225